In [2]:
# set up the config
class Config:
    BATCH_SIZE = 8
    MAX_LEN = 64
    TARGET = 'label'
    TEXT = 'text'
    MODEL = 'distilbert-base-multilingual-cased'
    #MODEL = 'bert-base-multilingual-cased'
    #MODEL = 'xlm-roberta-base'
    LEARNING_RATE = 1e-05
    EPOCHS = 5
    EPS = 1e-08
    random_seed = 0xfeedbeef
    dataset = "EENLP.NLI"
    train_on = 'russian'


In [ ]:
%%capture
# prepare env

!pip install transformers
!pip install wget
!pip install urllib2
!pip install wandb -qqq
!pip install jsonlines

In [ ]:
import json
import random

import torch
import wandb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification 
from transformers import get_linear_schedule_with_warmup, AdamW

In [ ]:
df = pd.read_csv('NLI.csv')
df['text'] = '[CLS] ' + df['sentence1'] + ' [SEP] ' + df['sentence2'] 


test_df = df[df['language']!=Config.train_on]
train_df = df[df['language']==Config.train_on]


add_to_test = train_df.sample(round(len(train_df)*.15))

test_df = test_df.append(add_to_test)
train_df = train_df.drop(add_to_test.index)

In [ ]:
# Log in to your W&B account
wandb.login()
wandb.init(
      project="nli", 
      # Track hyperparameters and run metadata
      config=dict([(k,v) for k,v in Config.__dict__.items() if k[0]!='_']),
      reinit=True
)
wandb.run.name += Config.train_on.upper() + '-TRAIN'+f'_{Config.MODEL}'
wandb.run.save()    

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

wandb: Currently logged in as: norest (use `wandb login --relogin` to force relogin)


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


device(type='cuda')

In [ ]:
label_encoder = None
labels_codes = None

def load_dataset(fn):
    return pd.read_json(fn, lines=True) 

def load_dataset_and_split(fn, fraction=.8):
    data = pd.read_json(fn, lines=True)
    df_train=data.sample(frac=fraction,random_state=200)
    df_test=data.drop(df_train.index).reset_index(drop=True)
    df_train = df_train.reset_index(drop=True)
    return df_train, df_test

def process_dataset(data, tokenizer, seq=False):
    global label_encoder, labels_codes
    if label_encoder is None:
        print('init of label encoder')
        label_encoder = LabelEncoder().fit(data[Config.TARGET])
        keys = list(sorted(set(data[Config.TARGET])))
        labels = label_encoder.transform(keys)
        labels_codes = dict(zip(keys, labels))
    data[Config.TARGET] = label_encoder.transform(data[Config.TARGET])

    input_ids = torch.tensor([])
    attention_masks = torch.tensor([])

    for sent in data.loc[:, Config.TEXT]:
        encoded_sent = tokenizer.encode_plus(sent, add_special_tokens = True,
                                             max_length = Config.MAX_LEN, 
                                             padding = 'max_length',
                                             pad_to_max_length=True,
                                             truncation = True,
                                             return_tensors = 'pt')
        input_ids = torch.cat([input_ids, encoded_sent['input_ids']])
        attention_masks = torch.cat([attention_masks, encoded_sent['attention_mask']])
    labels = torch.tensor(data[Config.TARGET].values)
    dataset = TensorDataset(input_ids, attention_masks, labels)
    if seq:
        return DataLoader(dataset, sampler = SequentialSampler(dataset), batch_size = Config.BATCH_SIZE)
    else:
        return DataLoader(dataset, sampler = RandomSampler(dataset), batch_size = Config.BATCH_SIZE)


In [ ]:
# fix PRNG
random.seed(Config.random_seed)
np.random.seed(Config.random_seed)
torch.manual_seed(Config.random_seed)
torch.cuda.manual_seed_all(Config.random_seed)

# init tokenizer
tokenizer = AutoTokenizer.from_pretrained(Config.MODEL, truncation=True, do_lower_case=False)

# split english dataset
print('parsing and preparing data, it will take a while.')
print(train_df.language.iloc[0], end='... \t')
#train_df, test_df = load_dataset_and_split(Config.full_data, .8)
train_loader = process_dataset(train_df, tokenizer, seq=False)

#eval_loaders['english'] = process_dataset(test_df, tokenizer, seq=True)
#print('done.')
# prepare eval for other language
eval_loaders = dict()
for lang in test_df.language.drop_duplicates().tolist():
    print(lang, end='... \t')
    eval_lang_df = test_df[test_df.language==lang]
    eval_loaders[lang] = process_dataset(eval_lang_df, tokenizer, seq=True)
    print('done.')



Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

parsing and preparing data, it will take a while.
polish... 	init of label encoder
bulgarian... 	

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


done.
english... 	done.
russian... 	done.
slovene... 	done.
polish... 	done.


In [ ]:
len(labels_codes)

3

In [ ]:
# Init model
model = AutoModelForSequenceClassification.from_pretrained(
   Config.MODEL,
   num_labels = len(labels_codes),
   output_attentions = False,
   output_hidden_states = False    
)
model.to(device)

Downloading:   0%|          | 0.00/517M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['pre_classifier.bias', 'classifie

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
      

In [ ]:
def train(model, epoch, loader, scheduler=None):
    model.train()

    train_loss_accum = 0
    fin_targets=[]
    fin_outputs=[]
    
    for index, (sentence, attention_mask, label) in tqdm(enumerate(loader)):
        model.zero_grad()

        sentence = sentence.to(device).long()
        attention_mask = attention_mask.to(device).long()
        label = label.to(device).long()

        output = model(sentence, attention_mask = attention_mask, labels = label)
        loss_value, logits = output[0], output[1]
        train_loss_accum += loss_value.item()
        fin_targets.extend(label.cpu().detach().numpy().tolist())
        logits = logits.cpu().detach().numpy()
        fin_outputs.extend(np.argmax(logits, axis=1))
        
        loss_value.backward()
        optimizer.step()
        if scheduler:
            scheduler.step()

    avg_loss = train_loss_accum / index
    train_accuracy = metrics.accuracy_score( fin_targets, fin_outputs )
    train_f1_micro = metrics.f1_score(fin_targets, fin_outputs, average='micro')
    train_f1_macro = metrics.f1_score(fin_targets, fin_outputs, average='macro')

    wandb.log({"train/loss": avg_loss, 
               "train/acc":  train_accuracy,
               "train/f1_micro" : train_f1_micro,
               "train/f1_macro" : train_f1_macro,
               "epoch":epoch,
              })



In [ ]:
def validation(model, testing_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for sentence, attention_mask, targets in testing_loader:
            sentence = sentence.to(device).long()
            attention_mask = attention_mask.to(device).long()
            outputs = model(sentence, attention_mask = attention_mask)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            logits = outputs.logits.cpu().detach().numpy()
            fin_outputs.extend(np.argmax(logits, axis=1))
#             break
    return fin_targets, fin_outputs

def eval_model(model, epoch=-1):
    for lang, eval_loader in eval_loaders.items():
        targets, preds = validation(model, eval_loader)
        scores = dict()
        scores[f'valid/acc/{lang}'] = metrics.accuracy_score( targets, preds )
        scores[f'valid/f1_micro/{lang}'] = metrics.f1_score(targets, preds, average='micro')
        scores[f'valid/f1_macro/{lang}'] = metrics.f1_score(targets, preds, average='macro')
        scores['epoch'] = epoch
        print(scores)
        wandb.log(scores)


In [ ]:
optimizer = AdamW(model.parameters(), lr = Config.LEARNING_RATE, eps = Config.EPS)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, 
                                            num_training_steps = Config.EPOCHS*len(train_df)/Config.BATCH_SIZE)

eval_model(model, epoch=-1)
for epoch in range(Config.EPOCHS):
    train(model, epoch, train_loader, scheduler)
    eval_model(model, epoch)


{'valid/acc/bulgarian': 0.3334666666666667, 'valid/f1_micro/bulgarian': 0.3334666666666667, 'valid/f1_macro/bulgarian': 0.16719914802981894, 'epoch': -1}
{'valid/acc/english': 0.33293333333333336, 'valid/f1_micro/english': 0.33293333333333336, 'valid/f1_macro/english': 0.17460334717215778, 'epoch': -1}
{'valid/acc/russian': 0.3336, 'valid/f1_micro/russian': 0.3336, 'valid/f1_macro/russian': 0.16771367666447753, 'epoch': -1}
{'valid/acc/slovene': 0.477124183006536, 'valid/f1_micro/slovene': 0.477124183006536, 'valid/f1_macro/slovene': 0.2153392330383481, 'epoch': -1}
{'valid/acc/polish': 0.08148148148148149, 'valid/f1_micro/polish': 0.08148148148148149, 'valid/f1_macro/polish': 0.05022831050228311, 'epoch': -1}


957it [02:30,  6.38it/s]


{'valid/acc/bulgarian': 0.37093333333333334, 'valid/f1_micro/bulgarian': 0.37093333333333334, 'valid/f1_macro/bulgarian': 0.2780246103415916, 'epoch': 0}
{'valid/acc/english': 0.36573333333333335, 'valid/f1_micro/english': 0.36573333333333335, 'valid/f1_macro/english': 0.26460522418855664, 'epoch': 0}
{'valid/acc/russian': 0.3641333333333333, 'valid/f1_micro/russian': 0.3641333333333333, 'valid/f1_macro/russian': 0.26140044174832017, 'epoch': 0}
{'valid/acc/slovene': 0.1503267973856209, 'valid/f1_micro/slovene': 0.1503267973856209, 'valid/f1_macro/slovene': 0.1577458729745311, 'epoch': 0}
{'valid/acc/polish': 0.9022222222222223, 'valid/f1_micro/polish': 0.9022222222222223, 'valid/f1_macro/polish': 0.8554126987042728, 'epoch': 0}


957it [02:28,  6.43it/s]


{'valid/acc/bulgarian': 0.3948, 'valid/f1_micro/bulgarian': 0.3948, 'valid/f1_macro/bulgarian': 0.3248694307903787, 'epoch': 1}
{'valid/acc/english': 0.3778666666666667, 'valid/f1_micro/english': 0.37786666666666674, 'valid/f1_macro/english': 0.29572965999366635, 'epoch': 1}
{'valid/acc/russian': 0.3874666666666667, 'valid/f1_micro/russian': 0.38746666666666674, 'valid/f1_macro/russian': 0.31125862771899365, 'epoch': 1}
{'valid/acc/slovene': 0.2222222222222222, 'valid/f1_micro/slovene': 0.2222222222222222, 'valid/f1_macro/slovene': 0.21417780823340968, 'epoch': 1}
{'valid/acc/polish': 0.92, 'valid/f1_micro/polish': 0.92, 'valid/f1_macro/polish': 0.8874712034452985, 'epoch': 1}


...

{'valid/acc/bulgarian': 0.38306666666666667, 'valid/f1_micro/bulgarian': 0.38306666666666667, 'valid/f1_macro/bulgarian': 0.3133037945008437, 'epoch': 2}
{'valid/acc/english': 0.38, 'valid/f1_micro/english': 0.38, 'valid/f1_macro/english': 0.3018651915878768, 'epoch': 2}
{'valid/acc/russian': 0.3761333333333333, 'valid/f1_micro/russian': 0.3761333333333333, 'valid/f1_macro/russian': 0.29282344717991765, 'epoch': 2}
{'valid/acc/slovene': 0.19607843137254902, 'valid/f1_micro/slovene': 0.19607843137254902, 'valid/f1_macro/slovene': 0.2010597605340164, 'epoch': 2}
{'valid/acc/polish': 0.9237037037037037, 'valid/f1_micro/polish': 0.9237037037037037, 'valid/f1_macro/polish': 0.8954508434253318, 'epoch': 2}


957it [02:28,  6.42it/s]


{'valid/acc/bulgarian': 0.3928, 'valid/f1_micro/bulgarian': 0.3928, 'valid/f1_macro/bulgarian': 0.3361829538898731, 'epoch': 3}
{'valid/acc/english': 0.3916, 'valid/f1_micro/english': 0.3916, 'valid/f1_macro/english': 0.3318669838549942, 'epoch': 3}
{'valid/acc/russian': 0.3844, 'valid/f1_micro/russian': 0.3844, 'valid/f1_macro/russian': 0.3193372411636626, 'epoch': 3}
{'valid/acc/slovene': 0.24509803921568626, 'valid/f1_micro/slovene': 0.24509803921568626, 'valid/f1_macro/slovene': 0.2366541268830391, 'epoch': 3}
{'valid/acc/polish': 0.9177777777777778, 'valid/f1_micro/polish': 0.9177777777777778, 'valid/f1_macro/polish': 0.8859413573419649, 'epoch': 3}


957it [02:28,  6.44it/s]


{'valid/acc/bulgarian': 0.39813333333333334, 'valid/f1_micro/bulgarian': 0.39813333333333334, 'valid/f1_macro/bulgarian': 0.3494810308546102, 'epoch': 4}
{'valid/acc/english': 0.39653333333333335, 'valid/f1_micro/english': 0.3965333333333333, 'valid/f1_macro/english': 0.3452098349250791, 'epoch': 4}
{'valid/acc/russian': 0.38693333333333335, 'valid/f1_micro/russian': 0.38693333333333335, 'valid/f1_macro/russian': 0.326767361810631, 'epoch': 4}
{'valid/acc/slovene': 0.26143790849673204, 'valid/f1_micro/slovene': 0.26143790849673204, 'valid/f1_macro/slovene': 0.24768370021626804, 'epoch': 4}
{'valid/acc/polish': 0.9222222222222223, 'valid/f1_micro/polish': 0.9222222222222223, 'valid/f1_macro/polish': 0.8918821380912618, 'epoch': 4}
